In [1]:
!pip install onnx
#imports
import math
#PyTorch imports
import torch
import torch.nn as nn
from torch.optim import AdamW
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.utils.data import ConcatDataset

#Import SciPy to load the Flowers-102 dataset from the .mat format
import scipy.io

#Other imports
import numpy as np
import matplotlib.pyplot as plt

#Import time so that we can train the model within the 12 hour time limit
import time

In [2]:
#Define transforms
trainingTransform = transforms.Compose([
    transforms.Resize(224), #Change this value to edit the image size
    transforms.RandomRotation(90), #Rotates each image by a random angle between 0-90 degrees
    transforms.CenterCrop(224), #Must be the same as resize value. This is used to ensure the image is square
    #Default probability for both flips is p=0.5, can be changed if needed
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(0,1) #zerocenter normalization
])

trainingTransformBrightness = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomRotation(90),
    transforms.CenterCrop(224),
    transforms.ColorJitter(contrast=(0.5, 0.9), brightness=0.05),
    transforms.ToTensor(),
    transforms.Normalize(0,1)
])

trainingTransformSaturation = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomRotation(90),
    transforms.CenterCrop(224),
    transforms.ColorJitter(saturation=(0.5, 0.9), hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(0,1)
])

nonTrainTransform = transforms.Compose([
    #Same as training transform without flipping as required
    #by assessment rules
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(0,1)
])

#Load Flowers-102 dataset and split it into train, validate and test splits
trainingSet = datasets.Flowers102(root="./flowers", split="train", download=True, transform=trainingTransform)
trainingSet2 = datasets.Flowers102(root="./flowers", split="train", download=True)
trainingSetBright, trainingSetSat = random_split(trainingSet2, (510, 510)) #splits the data set into non-overlapping halfs
trainingSetSat.dataset.transform = trainingTransformSaturation
trainingSetBright.dataset.transform = trainingTransformBrightness
trainingSet = ConcatDataset([trainingSet, trainingSetBright, trainingSetBright])

validationSet = datasets.Flowers102(root="./flowers", split="val", download=True, transform=nonTrainTransform)
testingSet = datasets.Flowers102(root="./flowers", split="test", download=True, transform=nonTrainTransform)

In [ ]:
#Checking the splits are the correct length
print(len(trainingSet))
print(len(validationSet))
print(len(testingSet))

In [ ]:
#Checking the training transformations are ok
figure = plt.figure(figsize=(10,10))
col, row = 2, 2
for i in range(1, 5): #TODO: Change this to be rows * columns
  randImg = torch.randint(len(trainingSet), size=(1,)).item() #Size defines the shape of the tensor
  img, label = trainingSet[randImg]
  figure.add_subplot(row, col, i)
  plt.title(label)
  plt.axis(False)
  plt.imshow(img.squeeze().permute(1,2,0)) #Permute gets the dimentions in the correct order
plt.show()

In [3]:
#Checking if collab can connect to a GPU
if torch.cuda.is_available():
  device = torch.device('cuda:0')
else:
  device = torch.device('cpu')
print(device)


cuda:0


In [4]:
#The CNN itself
class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    #DEFINING THE LAYERS. THIS IS WHERE WE WILL SPEND MOST OF OUR TIME
    self.netLayers = nn.Sequential(
        # block 1 
        nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(64),

        nn.MaxPool2d(2,2),
        nn.BatchNorm2d(64),
 
        #block 2
        
        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=(1,1), padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(128),

        nn.MaxPool2d(2,2),
        nn.BatchNorm2d(128),

        #block3
        nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=(1,1), padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(256),

        nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=(1,1), padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(256),

        nn.MaxPool2d(2,2),
        nn.BatchNorm2d(256),
        
        
        #block 4
        nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=(1,1), padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(512),
        
        nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=(1,1), padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(512),

        nn.MaxPool2d(2,2),
        nn.BatchNorm2d(512),

        #block 5
        nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=(1,1), padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(512),

        nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=(1,1), padding=1),
        nn.BatchNorm2d(512),

        nn.MaxPool2d(2,2),
        nn.BatchNorm2d(512),

        nn.Flatten(),

        # fully connected layer
        nn.LazyLinear(4096),
        nn.ReLU(),
        nn.BatchNorm1d(4096),
        nn.Dropout(0.8),

        nn.LazyLinear(4096),
        nn.ReLU(),
        nn.BatchNorm1d(4096),
        nn.Dropout(0.8),

        nn.LazyLinear(1000),
        nn.ReLU(),
        nn.BatchNorm1d(1000),
        nn.Dropout(0.5),

        nn.Linear(1000, 102), #102 for the classifiers
    )
  
  def forward(self, x):
      result = self.netLayers(x)
      #print(result.shape)
      return result
  

print(CNN())


CNN(
  (netLayers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 

c:\Users\Inari\anaconda3\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [5]:
#Code to train the CNN based on a set number of epochs
def epochCountTrain(trainLoader, cnn, optimizer, lossFunction):
  totalLoss, correct = 0, 0
  size = len(trainLoader.dataset)
  for i, data in enumerate(trainLoader, 0):
    input, labels = data
    input = input.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    output = cnn(input)
    loss = lossFunction(output, labels)
    loss.backward()
    optimizer.step()
    totalLoss += loss.item()
    correct += (output.argmax(1)==labels).sum().item()
    if i % 20 == 0:
      loss, currentLoss = loss.item(), (i + 1) * len(input)
      print(f"loss: {loss:>7f}    [{currentLoss:>5d}/{size:>5d}]")
      
  epochTrainLoss.append(float(loss))
  epochTrainAccuracy.append(round((correct/size)*100, 1))
  print(f"\nAvg train loss: {(totalLoss/len(trainLoader)):>8f}")
  print(f"Train accuracy = {((correct/size) * 100): >0.1f}%")

In [6]:
#Code to guess accuracy based on validation set
def validation(valLoader, cnn, lossFunction):
  valSize = len(valLoader.dataset)
  correct = 0
  loss = []
  with torch.no_grad(): #as we are not calling backward, use no grad to save memory
    for batch in valLoader:
      input, labels = batch
      input = input.to(device)
      labels = labels.to(device)
      prediciton = cnn(input)
      loss.append(lossFunction(prediciton, labels).item())
      correct += (labels == prediciton.argmax(1)).sum().item()
  
  epochValLoss.append((np.average(loss)))
  epochValAccuracy.append(round((correct/valSize)*100, 1))
  print(f"Accuracy on validation set = {((correct / valSize) * 100):>0.1f}%")
  print(f"Average loss on validation set = {(np.average(loss))}\n")




In [7]:
#Code that gets the accuracy of the test set!
def test(testLoader, cnn):
  testSize = len(testLoader.dataset)
  correct = 0
  top5 = 0 
  with torch.no_grad():
    for batch in testLoader:
      input, labels = batch
      input = input.to(device)
      labels = labels.to(device)
      prediciton = cnn(input)
      correct +=(prediciton.argmax(dim=1) == labels).sum().item()
      #top5 += sum((labels in prediciton.topk(5, dim=1)))
  print("----------------------------------------------------")    
  print(f"Accuracy on test set = {((correct / testSize) * 100):>0.1f}%")
  #print(f"top5: {((top5 / testSize) * 100):>0.1f}%")
  print("----------------------------------------------------")  


In [8]:
#Creating a new CNN
model = CNN().to(device)

In [20]:
#Loading a CNN
PATH = "./mp510.pth" #Change to wherever the .pth file is located
model = CNN()
model.load_state_dict(torch.load(PATH))
model = model.to(device)

In [13]:
#Lists for graphs
epochTrainLoss = []
epochTrainAccuracy = []

epochValLoss = []
epochValAccuracy = []

epochCountList = []

In [10]:
#Instantiate the model, loss function and optimiser, done to prevent resets when rerunning the code below
epochCount = 1
learningRate = 0.00005
batchSize = 32
lossFunction = nn.CrossEntropyLoss()

optimizer = AdamW(lr=learningRate, params=model.parameters(), weight_decay=0.1)
trainDataloader = DataLoader(trainingSet, batch_size=batchSize, shuffle=True)
valDataloader = DataLoader(validationSet, batch_size=batchSize, shuffle=True)
testDataloader = DataLoader(testingSet, batch_size=batchSize, shuffle=True)



In [17]:
#Actual code to get it all running
for i in range(epochCount):
  print(f"Epoch {len(epochCountList)+1}")
  model.train()
  epochCountTrain(trainDataloader, model, optimizer, lossFunction)
  model.eval()
  validation(valDataloader, model, lossFunction)
  epochCountList.append(len(epochCountList) + 1)

Epoch 2
loss: 0.399134    [   32/ 2040]
loss: 0.518277    [  672/ 2040]
loss: 0.264113    [ 1312/ 2040]
loss: 0.478978    [ 1952/ 2040]

Avg train loss: 0.371001
Train accuracy = 91.3%
Accuracy on validation set = 54.3%
Average loss on validation set = 2.0986083298921585



In [ ]:
#plot the loss and accuracy of the training and validation set
plt.plot(epochCountList, epochTrainLoss, label="Training loss")
plt.plot(epochCountList, epochValLoss, label="Validation loss")
plt.title("Train & validation loss")
plt.legend()
plt.show()

plt.plot(epochCountList, epochTrainAccuracy, label="Training accuracy")
plt.plot(epochCountList, epochValAccuracy, label="Validation accuracy")
plt.title("Train & validation accuracy")
plt.legend()
plt.show()

In [ ]:
#Save the model to the drive
from google.colab import drive
drive.mount('/content/gdrive')

In [19]:
PATH ="./mp510.pth" #EDIT TO DEFINE WHERE TO SAVE
torch.save(model.state_dict(), PATH)

In [21]:
#Run the model on the test set
test(testDataloader, model)

----------------------------------------------------
Accuracy on test set = 43.6%
----------------------------------------------------


In [ ]:
for test_images, _ in trainDataloader:  
  torch.onnx.export(model, test_images.to(device), "alexnet.onnx", verbose=True)
  break